# 15 - Keras: Clasificación de textos con Redes Neuronales  (Perceptron Multicapa)

* En este notebook vamos a ver como crear un modelo que clasifique las frases según su polaridad (Positiva o Negativa) con una red neuronal, en particular con un ***Perceptrón Multicapa***.


* Este ejemplo resuelve el mismo problema que el notebook "*08_NLTK_Clasificacion_Textos_Naive_Bayes.ipynb*" para que puedan compararse los resultados.



* En este notebook vamos a tener dos conjuntos de frases clasificadas según su polaridad:
    1. **train**: Conjunto de frases clasificadas para entrenar la red.
    3. **Test**: Conjunto de frases de test a predecir por la red.
    
    
* Los pasos que vamos a dar son los siguientes:

    1. Normalización de las frases.
    2. Bolsa de palabras y codificación del target
    3. Definición de la Red Neuronal
    4. Entrenamiento de la Red Neuronal
    5. Evaluación del modelo

<hr>


# Frases a clasificar

In [1]:
train = [('I love this car', 'positive'),
         ('This view is amazing', 'positive'),
         ('I feel great this morning', 'positive'),
         ('I am so excited about the concert', 'positive'),
         ('He is my best friend', 'positive'),
         ('Going well', 'positive'),
         ('Thank you', 'positive'),
         ('Hope you are doing well', 'positive'),
         ('I am very happy', 'positive'),
         ('Good for you', 'positive'),
         ('It is all good. I know about it and I accept it.', 'positive'),
         ('This is really good!', 'positive'),
         ('Tomorrow is going to be fun.', 'positive'),
         ('These are great apples today.', 'positive'),
         ('How about them apples? Thomas is a happy boy.', 'positive'),
         ('I love this sandwich.', 'positive'),
         ('This is an amazing place!', 'positive'),
         ('I feel very good about these beers.', 'positive'),
         ('This is my best work.', 'positive'),
         ('What an awesome view', 'positive'),
         ('I do not like this car', 'negative'),
         ('This view is horrible', 'negative'),
         ('I feel tired this morning', 'negative'),
         ('I am not looking forward to the concert', 'negative'),
         ('He is my enemy', 'negative'),
         ('I am a bad boy', 'negative'),
         ('This is not good', 'negative'),
         ('I am bothered by this', 'negative'),
         ('I am not connected with this', 'negative'),
         ('Sadistic creep you ass. Die.', 'negative'),
         ('All sorts of crazy and scary as hell.', 'negative'),
         ('Not his emails, no.', 'negative'),
         ('His father is dead. Returned obviously.', 'negative'),
         ('He has a bomb.', 'negative'),
         ('Too fast to be on foot. We cannot catch them.', 'negative'),
         ('I do not like this restaurant', 'negative'),
         ('I am tired of this stuff.', 'negative'),
         ("I can't deal with this", 'negative'),
         ('He is my sworn enemy!', 'negative'),
         ('My boss is horrible.', 'negative')]

test = [('I feel happy this morning', 'positive'),
        ('Larry is my friend', 'positive'),
        ('I do not like that man', 'negative'),
        ('My house is not great', 'negative'),
        ('Your song is annoying', 'negative'),
        ('The beer was good.', 'positive'),
        ('I do not enjoy my job', 'negative'),
        ("I feel amazing!", 'positive'),
        ('Gary is a friend of mine.', 'positive'),
        ("I can't believe I'm doing this.", 'negative')]

<hr>


## Normalización

* En primer lugar vamos a pasar a normalizar las frases. Para ello realizaremos lo siguiente:

    - Eliminamos los signos de puntuación
    - Pasamos el texto a minúsculas
    - Eliminamos las Stop-Words
    - Eliminamos las palabras con menos de 3 caracteres
    
* Nos creamos una función que realice este procesamiento para las frases dadas.


***NOTA***: *Para este ejemplo en particular se hace una normalización muy básica pero suficiente para realizar este ejemplo con caracter didáctico*.

In [2]:
import warnings
warnings.filterwarnings("ignore")
import re
from nltk.corpus import stopwords

def normalize(sentenses):
    """normalizamos la lista de frases"""
    sen = []
    for (words, sentiment) in sentenses:
        words_filtered = []
        for word in words.split():
            # Eliminamos signos de puntuación y lo pasamos a minusculas
            word = re.sub(r'[^\w\s]', '', word).lower()
            # Filtramos stop words y las palabras con menos de 3 caracteres
            if len(word) > 2 and word not in stopwords.words():
                words_filtered.append(word)
        sen.append(words_filtered)
    return sen

* Pasamos a normalizar los conjuntos de entrenamiento y test.


* Obtenemos en una lista el target de los conjunto de entrenamiento y test.

In [3]:
X_train = normalize(train)
y_train = [sentiment for (words, sentiment) in train]
X_test = normalize(test)
y_test = [sentiment for (words, sentiment) in test]

<hr>


# Bolsa de palabras y codificación del target

* Cuando trabajamos con redes neuronales para la clasificación de textos es necesario tener:
    1. ***Input***: Un vector por documento en el que en cada posición del vertor represente el peso que tiene la palabra en ese documento bien sea por frecuencia, por aparición de la palabra en el documento (One-Hot-Encode) o por su TF-IDF.
    2. ***Output***: Al resolver la red neuronal un problema de clasificación de textos, la salida de la red tiene que ser un valor numérico, por lo que el target hay que codificalo en valores numéricos. En este ejemplo al ser una clasificación binaria (positivo o negativo) codificaremos estos dos valores en '0' y '1'.
    
    
* A continuación pasamos a crear la bolsa de palabras.


* Para realizar esta labor vamos a usar la clase '***Tokenizer***' de Keras: https://keras.io/preprocessing/text/


* Lo que vamos a hacer es crear una ***bolsa de palabras de frecuencias*** cogiendo las 'N' palabras más frecuentes del Corpus. En este caso el Corpus tiene 63 palabras por lo que vamos a seleccionar todas las palabras.


* Es muy importante saber la ***dimensión (longitud) del vector de palabras*** que representará al documento ya que esa dimensión será el ***número de neuronas de entrada*** (+1) que tendrá nuestra red neuronal.


* La bolsa de palabras la vamos a crear como una matriz llamando a la función '***text_to_matrix***':http://faroit.com/keras-docs/2.0.2/preprocessing/text/


In [4]:
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils


num_words = 64

# Creamos un objeto de la clase Tokenizer indicandole el número de palabras
tokenizer = Tokenizer(num_words=num_words)

# Calculamos la bolsa de palabras (generamos el vocabulario)
tokenizer.fit_on_texts(X_train)

# Creamos una matriz de bolsa de palabras donde:
#     Fila: Representa a un documento
#     Columna: Representa a una palabra
X_matrix_train = tokenizer.texts_to_matrix(X_train, mode='count')
X_matrix_test = tokenizer.texts_to_matrix(X_test, mode='count')

print('Dimensión de la matriz de entrenamiento: {dim}\n'.format(dim=X_matrix_train.shape))

print('Bolsa de palabras de los datos de entrenamiento:\n{bolsa}\n'.format(bolsa=tokenizer.word_counts))

print('Índice de las palabras en la Matriz:\n{index}\n'.format(index=tokenizer.word_index))

print('Matriz de entrenamiento:\n{matrix}\n'.format(matrix=X_matrix_train))

Using TensorFlow backend.


Dimensión de la matriz de entrenamiento: (40, 64)

Bolsa de palabras de los datos de entrenamiento:
OrderedDict([('love', 2), ('car', 2), ('view', 3), ('amazing', 2), ('feel', 3), ('great', 2), ('morning', 2), ('excited', 1), ('concert', 2), ('best', 2), ('friend', 1), ('going', 2), ('well', 2), ('thank', 1), ('hope', 1), ('happy', 2), ('good', 5), ('know', 1), ('accept', 1), ('really', 1), ('tomorrow', 1), ('fun', 1), ('apples', 2), ('today', 1), ('thomas', 1), ('boy', 2), ('sandwich', 1), ('place', 1), ('beers', 1), ('work', 1), ('awesome', 1), ('like', 2), ('horrible', 2), ('tired', 2), ('looking', 1), ('forward', 1), ('enemy', 2), ('bad', 1), ('bothered', 1), ('connected', 1), ('sadistic', 1), ('creep', 1), ('ass', 1), ('sorts', 1), ('crazy', 1), ('scary', 1), ('hell', 1), ('emails', 1), ('father', 1), ('dead', 1), ('returned', 1), ('obviously', 1), ('bomb', 1), ('fast', 1), ('foot', 1), ('cannot', 1), ('catch', 1), ('restaurant', 1), ('stuff', 1), ('cant', 1), ('deal', 1), ('sworn

* Lo siguiente que vamos a hacer en codificar la salida
    - Negativo: 0
    - Positivo: 1
    

* Esto lo vamos a hacer con la clase '***LabelEncoder***' de scikit: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
y_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

<hr>


# Definición de la Red Neuronal


* Para clasificar los textos en positivo o negativo vamos a crear una red con la siguiente arquitectura:

    - ***Capa 1***: Capa de entrada con 64 Neuronas (una por palabra del corpus)
    - ***Capa 2***: 20 Neuronas con una función de activación 'relu'
    - ***Capa 3***: 5 Neuronas con una función de activación 'relu'
    - ***Capa 4***: Capa de salida con 1 neurona y una función de activación 'sigmoidal'
    
    
* Para las capas 2 y 3 vamos a poner un dropout del 10% para que nuestra red no se sobreajuste y generalize mejor.
    
    
* Por último utilizaremos:
    - Función de perdida: 'binary_crossentropy' (Funciones de Perdida: https://keras.io/losses/)
    - Optimizador: 'adam' (Optimizadores: https://keras.io/optimizers/)
    - Metricas: en cada epoch pediremos que nos muestre el accuracy (Métricas: https://keras.io/metrics/)
    


In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics

np.random.seed(9)
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=num_words))
model.add(Dropout(0.1))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

* Una vez creada la red mostramos su arquitectura:
    - 64 Neuronas de entrada
    - Capa 2: 20 neuronas
    - Conexiones "capa de entrada -> Capa 2" = (64*20) + 20 = 1300 Conexiones
    - Capa 3: 5 neuronas
    - Conexiones "Capa 2 -> Capa 3" = (20*5) + 5 = 105 Conexiones
    - Capa de salida: 1 Neurona
    - Conexiones "Capa 3 -> Capa de salida" = (5*1) + 1 = 6 Conexiones

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 105       
_________________________________________________________________
dropout_2 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 1,411
Trainable params: 1,411
Non-trainable params: 0
_________________________________________________________________


<hr>


# Entrenamos la red


* Una vez definida la Red Nuronal y compilada ya podemos entrenarla (ajustar los pesos de las conexiones).


* Para entrenar la Red Nuronal (model) vamos a llamar al método '***fit()***' que recibe los siguientes parámetros:

    - ***datos de entrada*** (X_matrix_train)
    - ***target*** (y_train)
    - ***batch_size***: Número de muestras por epoch que utilizará para actualizar los pesos
    - ***epochs***: Número de veces que recorrerá el dataset para entrenar la red
    - verbose: 3 tipos de verbosidad mientras entrena la red (0,1,2)
    - ***validation_split***: Porcentaje de los datos de entrenamiento que utiliza para validar el modelo en cada epoch.
    
    
* Como lo que estamos haciendo es un ejemplo didáctico vamos a poner los siguientes valores:
    - *batch_size = 1* : Como tenemos pocos datos, podemos ajustar los pesos de la Red Neuronal cada vez que se calcule una muestra.
        + batch_size con valores muy pequeños: En teoria la red neuronal estaría muy bien entrenada pero tardaría mucho tiempo en entrenarse.
        + batch_size con valores muy grandes: En teoria la red neuronal estaría peor entrenada que con valores pequeños pero tardaría menos tiempo en ejecutarse.
    - *epochs = 20* : Como es un dataset pequeño no es necesario entrenar la red muchas veces por eso ponemos un valor relativamente pequeño.
    - *validation_split = 0.1* : Al ser un dataset con 40 frases, elegiremos en cada pasada solo 4 frases para evaluarlas. 

In [8]:
model.fit(X_matrix_train, y_train, batch_size=1, epochs=20, verbose=2, validation_split=0.1)

Train on 36 samples, validate on 4 samples
Epoch 1/20
 - 1s - loss: 0.7013 - acc: 0.4167 - val_loss: 0.7007 - val_acc: 0.2500
Epoch 2/20
 - 0s - loss: 0.6844 - acc: 0.6111 - val_loss: 0.6970 - val_acc: 0.5000
Epoch 3/20
 - 0s - loss: 0.6710 - acc: 0.6944 - val_loss: 0.6972 - val_acc: 0.5000
Epoch 4/20
 - 0s - loss: 0.6539 - acc: 0.7778 - val_loss: 0.7023 - val_acc: 0.2500
Epoch 5/20
 - 0s - loss: 0.6470 - acc: 0.8056 - val_loss: 0.7011 - val_acc: 0.2500
Epoch 6/20
 - 0s - loss: 0.6322 - acc: 0.8333 - val_loss: 0.6984 - val_acc: 0.2500
Epoch 7/20
 - 0s - loss: 0.6381 - acc: 0.8056 - val_loss: 0.6993 - val_acc: 0.2500
Epoch 8/20
 - 0s - loss: 0.6178 - acc: 0.8333 - val_loss: 0.6978 - val_acc: 0.2500
Epoch 9/20
 - 0s - loss: 0.6056 - acc: 0.8611 - val_loss: 0.6876 - val_acc: 0.2500
Epoch 10/20
 - 0s - loss: 0.5771 - acc: 0.8611 - val_loss: 0.6820 - val_acc: 0.5000
Epoch 11/20
 - 0s - loss: 0.5638 - acc: 0.8611 - val_loss: 0.6681 - val_acc: 0.5000
Epoch 12/20
 - 0s - loss: 0.5434 - acc: 0.

<hr>


# Evaluamos el modelo


* Al tratarse de un ejemplo didáctico vamos a medir solamente el accuracy.


* Para ello utilizaremos el método '***evaluate()***' que pasandole los datos de entrenamiento (datos y target), predice los datos y nos calcula el accuracy entre otras métricas.

In [9]:
# Evaluación del modelo con los mismos datos de entrenamiento
scores = model.evaluate(X_matrix_test, y_test)
print("Porcentaje acierto: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))


10/10 [==============================] - 0s 501us/step
Porcentaje acierto: acc: 90.00%


* Llamando al método '***predict()***' y pasandole unos datos de entrada, nos devuelve el valor de la salida.

In [10]:
predict = model.predict(X_matrix_test)
print(predict)

[[0.7410232 ]
 [0.5993302 ]
 [0.17878544]
 [0.64687943]
 [0.4562172 ]
 [0.5991951 ]
 [0.4562172 ]
 [0.6883626 ]
 [0.5993302 ]
 [0.46643698]]


* Como lo que nos interesa en un problema de clasificación es la categoria en que nos clasifica el documento y no el valor de salida de la red neuronal, podemos obtener la categoria de cada predicción llamando al método '***predict_classes()***', pasandole el/los dato/s de entrada.


* Hay que tener cuidado ya que nos va a dar un número entero (0 o 1) ya que hemos realizado el LabelEncoder previamente y tenemos que hacer el "encodeado inverso":

In [11]:
# Predecimos los resultados del conjunto de test
y_predict = model.predict_classes(X_matrix_test)
for index, prediction in enumerate(y_predict):
    text = test[index][0]
    y_true = test[index][1]
    y_predict = encoder.inverse_transform(prediction)
    print('{i} - {text} \n\t Real: {y_true} - Predicción: {y_predict} - Acierto: {acierto}'
          .format(i=index, text=text, y_true=y_true, y_predict=y_predict[0], acierto = y_true==y_predict[0]))

0 - I feel happy this morning 
	 Real: positive - Predicción: positive - Acierto: True
1 - Larry is my friend 
	 Real: positive - Predicción: positive - Acierto: True
2 - I do not like that man 
	 Real: negative - Predicción: negative - Acierto: True
3 - My house is not great 
	 Real: negative - Predicción: positive - Acierto: False
4 - Your song is annoying 
	 Real: negative - Predicción: negative - Acierto: True
5 - The beer was good. 
	 Real: positive - Predicción: positive - Acierto: True
6 - I do not enjoy my job 
	 Real: negative - Predicción: negative - Acierto: True
7 - I feel amazing! 
	 Real: positive - Predicción: positive - Acierto: True
8 - Gary is a friend of mine. 
	 Real: positive - Predicción: positive - Acierto: True
9 - I can't believe I'm doing this. 
	 Real: negative - Predicción: negative - Acierto: True


<hr>


# Bonus Track - Keras -

* Keras es una librería en Python desarrollada por François Chollet (ingeniero de Google) que en esencia es un wrapper sobre TensorFlow y Theano. 


https://github.com/keras-team/keras


* TensorFlow y Theano son librería muy potentes pero tienen el “problema” que de son muy difíciles de utilizar, por lo que Keras empezó a desarrollarse por Chollet con el objetivo de crear una librería sencilla de utilizar para temas de Deep Learning.

* Los principios en los que se basa Keras son los siguientes:

    - ***Modularidad***: Una red neuronal se entiende como una secuencia o grafo de capas de neuronas.

    - ***Minimalismo***: La librería proporciona lo justo y necesario para crear y entrenar redes maximizando la legibilidad del código.

    - ***Extensibilidad***: Los nuevos componentes tienen que ser fáciles de añadirse a Keras.

    - ***Python***: Keras es Python nativo.


* Keras es en realidad un "*wrapper*" sobre otras librerías como:
    - Tensorflow
    - Theano
    - CNTK
    
    
* Utiliza una de estas 3 librerías para ejecutar las redes neuronales que definamos.


* El utilizar una librería u otra se debe de indicar en el fichero de configuración de queras que se encuentra en:


```
~/.keras/keras.json
```

* El contenido de este fichero por defecto es el siguiente, indicando en el ***backend*** la librería que utilizará:


```
{ "floatx": "float32",
  "epsilon": 1e-07,
  "backend": "tensorflow",
  "image_data_format": "channels_last"
} 
```


## Como Construir modelos en Keras


* Lo que a continuación se cuenta se hace de manera muy genérica pero podemos definir los pasos que se deben de dar para contruir una red neuronal con Keras:

    1. ***Definir el modelo***: crear el ***Sequential*** model y añadir las capas con su configuración (https://keras.io/models/sequential/).
    2. ***Compilar el modelo***: definir o especificar la función de pérdida y llamar al método ***compile()***.
    3. ***Ajustar (fit) el modelo***: entrenar el modelo con los datos de entrenamiento llamando al método ***fit()***.
    4. ***Realizar predicciones***: usando el modelo se pueden realizar predicciones con nuevos datos llamando a los métodos ***evaluate()*** o ***predict()***.
